In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklego.metrics import equal_opportunity_score
from sklego.metrics import p_percent_score
from sklearn.metrics import log_loss
from sklearn.utils.extmath import squared_norm
from moopt.scalarization_interface import scalar_interface, single_interface, w_interface
from moopt import monise
import numpy as np
from sklearn.metrics import log_loss, accuracy_score
import optuna, sklearn, sklearn.datasets
from fair_models import coefficient_of_variation, MOOLogisticRegression, FindCLogisticRegression, FindCCLogisticRegression
from fair_models import calc_reweight
from fair_models import FairScalarization, EqualScalarization
from sklearn.model_selection import KFold

import sys
sys.path.append("./MMFP/")
from MMPF.MinimaxParetoFair.MMPF_trainer import SKLearn_Weighted_LLR, APSTAR

In [2]:
def kfold_methods(dataset, fair_feature, pred_feature, val_size = 100, n_folds = 5):
    kfold = KFold(n_folds, True, 1)
    
    metrics = ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation']
    methods = ['LogReg', 'RegEqual', 'RegDemo', 'RegRewe', 'RegMoo', 'RegEqMoo']
    
    index = [sum([[method]*len(metrics) for method in methods],[]),
        sum([metrics for method in methods],[])]
    results_test = pd.DataFrame(index=index, columns=metrics)
    
    for metric in metrics:
        for method in methods:
            for i in range(4):
                results_test.loc[(method,metric),metrics[i]] = 0
    
    for i in range(n_folds):
    
        result = next(kfold.split(dataset), None)

        train = dataset.iloc[result[0]].drop("Unnamed: 0", axis=1)
        test = dataset.iloc[result[1]].drop("Unnamed: 0", axis=1)

        X = train.drop([pred_feature], axis=1)
        y = train[pred_feature]
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size)
        X_test = test.drop([pred_feature], axis=1)
        y_test = test[pred_feature]

        sample_weight = calc_reweight(X_train, y_train, fair_feature)

        fair_scalarization = FairScalarization(X_train, y_train, fair_feature)
        fair_moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, fair_scalarization)
        fair_moo_learn.tune()

        equal_scalarization = EqualScalarization(X_train, y_train, fair_feature)
        equal_moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, equal_scalarization)
        equal_moo_learn.tune()

        for metric in metrics:

            # RegLog
            reg_log = FindCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, metric=metric).tune()
            results_test.loc[('LogReg',metric),metrics[0]] += reg_log.score(X_test, y_test)/n_folds
            results_test.loc[('LogReg',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_log,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('LogReg',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_log, X_test)/n_folds
            results_test.loc[('LogReg',metric),metrics[3]] += coefficient_of_variation(reg_log, X_test, y_test)/n_folds

            # RegEqual
            reg_equal = FindCCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, metric=metric, 
                                                 base_model='equal').tune()
            results_test.loc[('RegEqual',metric),metrics[0]] += reg_equal.score(X_test, y_test)/n_folds
            results_test.loc[('RegEqual',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_equal,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('RegEqual',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_equal, X_test)/n_folds
            results_test.loc[('RegEqual',metric),metrics[3]] += coefficient_of_variation(reg_equal, X_test, y_test)/n_folds

            # RegDemo
            reg_demo = FindCCLogisticRegression(X_train, y_train, X_val, y_val,fair_feature, metric=metric, 
                                                base_model='demographic').tune()
            results_test.loc[('RegDemo',metric),metrics[0]] += reg_demo.score(X_test, y_test)/n_folds
            results_test.loc[('RegDemo',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_demo,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('RegDemo',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_demo, X_test)/n_folds
            results_test.loc[('RegDemo',metric),metrics[3]] += coefficient_of_variation(reg_demo, X_test, y_test)/n_folds

            # RegRewe
            reg_rewe = FindCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature,metric=metric, 
                                               sample_weight=sample_weight).tune()
            results_test.loc[('RegRewe',metric),metrics[0]] += reg_rewe.score(X_test, y_test)/n_folds
            results_test.loc[('RegRewe',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_rewe,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('RegRewe',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_rewe, X_test)/n_folds
            results_test.loc[('RegRewe',metric),metrics[3]] += coefficient_of_variation(reg_rewe, X_test, y_test)/n_folds
            
            # Minimax
            

            # RegMoo
            reg_moo = fair_moo_learn.tune(metric)
            results_test.loc[('RegMoo',metric),metrics[0]] += reg_moo.score(X_test, y_test)/n_folds
            results_test.loc[('RegMoo',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_moo,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('RegMoo',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_moo, X_test)/n_folds
            results_test.loc[('RegMoo',metric),metrics[3]] += coefficient_of_variation(reg_moo, X_test, y_test)/n_folds

            # RegEqMoo
            reg_eqmoo = equal_moo_learn.tune(metric)
            results_test.loc[('RegEqMoo',metric),metrics[0]] += reg_eqmoo.score(X_test, y_test)/n_folds
            results_test.loc[('RegEqMoo',metric),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_eqmoo,
                                                                                                                 X_test, y_test)/n_folds
            results_test.loc[('RegEqMoo',metric),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_eqmoo, X_test)/n_folds
            results_test.loc[('RegEqMoo',metric),metrics[3]] += coefficient_of_variation(reg_eqmoo, X_test, y_test)/n_folds

    return results_test

In [3]:
def kfold_ensemble(dataset, fair_feature, pred_feature, val_size = 100, n_folds = 5):
    kfold = KFold(n_folds, True, 1)
    
    metrics = ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation']
    ensembles = ['voting soft', 'voting hard', 'knorau', 'knorae']
    methods = ['RegMoo', 'RegEqMoo']
    
    index = [sum([[method]*len(ensembles) for method in methods],[]),
            sum([ensembles for method in methods],[])]
    results_test = pd.DataFrame(index=index, columns=metrics)
    
    for ensemble in ensembles:
        for method in methods:
            for i in range(len(metrics)):
                results_test.loc[(method,ensemble),metrics[i]] = 0
    
    for i in range(n_folds):
    
        result = next(kfold.split(dataset), None)

        train = dataset.iloc[result[0]].drop("Unnamed: 0", axis=1)
        test = dataset.iloc[result[1]].drop("Unnamed: 0", axis=1)

        X = train.drop([pred_feature], axis=1)
        y = train[pred_feature]
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size)
        X_test = test.drop([pred_feature], axis=1)
        y_test = test[pred_feature]

        fair_scalarization = FairScalarization(X_train, y_train, fair_feature)
        fair_moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, fair_scalarization)
        fair_moo_learn.tune()

        equal_scalarization = EqualScalarization(X_train, y_train, fair_feature)
        equal_moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, equal_scalarization)
        equal_moo_learn.tune()

        for ensemble in ensembles:

            # RegMoo
            reg_moo = fair_moo_learn.ensemble_model(ensemble)
            results_test.loc[('RegMoo',ensemble),metrics[0]] += reg_moo.score(X_test, y_test)/n_folds
            results_test.loc[('RegMoo',ensemble),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_moo,
                                                                                                     X_test, y_test)/n_folds
            results_test.loc[('RegMoo',ensemble),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_moo,
                                                                                                               X_test)/n_folds
            results_test.loc[('RegMoo',ensemble),metrics[3]] += coefficient_of_variation(reg_moo, X_test, y_test)/n_folds

            # RegEqMoo
            reg_eqmoo = equal_moo_learn.ensemble_model(ensemble)
            results_test.loc[('RegEqMoo',ensemble),metrics[0]] += reg_eqmoo.score(X_test, y_test)/n_folds
            results_test.loc[('RegEqMoo',ensemble),metrics[1]] += equal_opportunity_score(sensitive_column=fair_feature)(reg_eqmoo,
                                                                                                     X_test, y_test)/n_folds
            results_test.loc[('RegEqMoo',ensemble),metrics[2]] += p_percent_score(sensitive_column=fair_feature)(reg_eqmoo, 
                                                                                                                 X_test)/n_folds
            results_test.loc[('RegEqMoo',ensemble),metrics[3]] += coefficient_of_variation(reg_eqmoo, X_test, y_test)/n_folds

    return results_test

## 1. German

In [15]:
german_data = pd.read_csv("Datasets - Limpos/german.csv")
german_fair_feature = "Sex"

In [68]:
german_result_metrics = kfold_methods(german_data, "Sex", "Risk")

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/pyt

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

...

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered 

In [69]:
german_result_metrics

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy            0.59619          0.881732  0.837138    0.842382
         equal_opportunity  0.590476           0.88392  0.839122    0.751382
         p_percent          0.590476          0.889973  0.907642    0.770161
         c_variation        0.573333          0.984589  0.985182    0.566732
RegEqual accuracy           0.609524          0.880847  0.816545    0.820122
         equal_opportunity  0.580952          0.937143  0.964838    0.682935
         p_percent          0.580952          0.885714  0.970207    0.749817
         c_variation        0.571429          0.956693  0.971335    0.600283
RegDemo  accuracy           0.590476          0.828174     0.757    0.881578
         equal_opportunity  0.577143          0.971429  0.992308    0.542577
         p_percent          0.571429                 1         1    0.489898
         c_variation        0.571429                 1         1    0.489898
RegRewe  accuracy           0.605714          0.894845  0.849523    0.826829
         equal_opportunity  0.592381          0.931825  0.888312    0.820742
         p_percent          0.605714           0.89595  0.934865    0.816706
         c_variation        0.573333          0.984589  0.985182    0.566732
RegMoo   accuracy            0.60381            0.8711  0.844508    0.841992
         equal_opportunity  0.607619          0.908281  0.808359    0.851606
         p_percent          0.594286          0.905042  0.909016    0.825173
         c_variation        0.586667          0.765671  0.738323    0.696436
RegEqMoo accuracy           0.592381           0.89026  0.929772    0.724869
         equal_opportunity  0.580952          0.989062  0.973913    0.556347
         p_percent          0.571429                 1         1    0.489898
         c_variation        0.571429                 1         1    0.489898

In [79]:
german_result_ensemble = kfold_ensemble(german_data, "Sex", "Risk")

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/pyt

In [80]:
german_result_ensemble

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft  0.601905          0.955066  0.857936    0.817419
         voting hard  0.588571          0.954552  0.908897    0.833107
         knorau       0.594286           0.96435  0.895576    0.832333
         knorae        0.55619          0.741677  0.638301    0.991349
RegEqMoo voting soft  0.571429                 1         1    0.489898
         voting hard  0.571429                 1         1    0.489898
         knorau       0.571429          0.984545  0.984938    0.520127
         knorae       0.561905          0.743043   0.63715    0.958173

In [16]:
german_data = german_data.drop(['Unnamed: 0'], axis=1)
X = german_data.drop(['Risk'], axis=1)
y = german_data['Risk']

X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=200)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=100)

a_train = X_train['Sex']
a_val = X_val['Sex']

a_train[a_train==0] = -1
a_val[a_val==0] = -1

C = 10**2
model = SKLearn_Weighted_LLR(X_train.values, y_train.values,
                             a_train.values, X_val.values,
                             y_val.values, a_val.values,
                             C_reg=C)

mua_ini = np.ones(a_val.max() + 1)
mua_ini /= mua_ini.sum()
results = APSTAR(model, mua_ini, niter=200, max_patience=200, Kini=1,
                      Kmin=20, alpha=0.5, verbose=False)
mu_best_list = results['mu_best_list']

mu_best = mu_best_list[-1]
model.weighted_fit(X_train.values, y_train.values, a_train.values, mu_best)

accuracy_score(y_test, model.predict(X_test)), equal_opportunity_score(sensitive_column="Sex")(model, X_test, y_test), p_percent_score(sensitive_column="Sex")(model, X_test, y_test), coefficient_of_variation(model, X_test, y_test)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


patience counter: 1 total iterations: 201
-----------------------------------------


(0.625, 0.9846153846153847, 0.9391750922805329, 0.706306071178631)

## 2. LSAC

In [19]:
lsac_data = pd.read_csv("Datasets - Limpos/lsac.csv")
lsac_fair_feature = "race1"
lsac_pred_feature = "pass_bar"

In [71]:
lsac_result_metrics = kfold_methods(lsac_data, lsac_fair_feature, lsac_pred_feature, 4000)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, 

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encoun

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: Runtim

In [72]:
lsac_result_metrics

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy           0.953598          0.998513   0.99361    0.284511
         equal_opportunity   0.95301                 1         1     0.28584
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953304          0.999257  0.996805    0.285177
RegEqual accuracy           0.953647          0.999257  0.995699    0.284269
         equal_opportunity   0.95301                 1         1     0.28584
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953451          0.999257    0.9966    0.284787
RegDemo  accuracy           0.953696          0.998885  0.995117    0.284194
         equal_opportunity   0.95301                 1         1     0.28584
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953304          0.999628  0.997821     0.28512
RegRewe  accuracy           0.953304                 1  0.999653    0.285069
         equal_opportunity  0.953108                 1  0.999884    0.285583
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953304                 1  0.999653    0.285069
RegMoo   accuracy           0.953793          0.997398  0.990792    0.284159
         equal_opportunity  0.953108                 1  0.999361    0.285583
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953451          0.999257  0.996223    0.284787
RegEqMoo accuracy           0.953451          0.998513  0.994569    0.284897
         equal_opportunity   0.95301                 1         1     0.28584
         p_percent           0.95301                 1         1     0.28584
         c_variation        0.953206          0.999257  0.997444    0.285434

In [81]:
lsac_result_ensemble = kfold_ensemble(lsac_data, lsac_fair_feature, lsac_pred_feature, 4000)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib

In [82]:
lsac_result_ensemble

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft   0.95301                 1         1     0.28584
         voting hard  0.953059                 1  0.999681    0.285711
         knorau       0.953157          0.998885  0.997125    0.285622
         knorae       0.918013          0.903266  0.883078    0.405791
RegEqMoo voting soft   0.95301                 1         1     0.28584
         voting hard   0.95301                 1         1     0.28584
         knorau       0.953157                 1  0.999042    0.285454
         knorae       0.917817          0.901731   0.88176    0.406343

In [24]:
lsac_data = lsac_data.drop(['Unnamed: 0'], axis=1)
X = lsac_data.drop(lsac_pred_feature, axis=1)
y = lsac_data[lsac_pred_feature]

X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=4000)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=4000)

a_train = X_train[lsac_fair_feature]
a_val = X_val[lsac_fair_feature]

a_train[a_train==0] = -1
a_val[a_val==0] = -1

C = 10**2
model = SKLearn_Weighted_LLR(X_train.values, y_train.values,
                             a_train.values, X_val.values,
                             y_val.values, a_val.values,
                             C_reg=C)

mua_ini = np.ones(a_val.max() + 1)
mua_ini /= mua_ini.sum()
results = APSTAR(model, mua_ini, niter=200, max_patience=200, Kini=1,
                      Kmin=20, alpha=0.5, verbose=False)
mu_best_list = results['mu_best_list']

mu_best = mu_best_list[-1]
model.weighted_fit(X_train.values, y_train.values, a_train.values, mu_best)

accuracy_score(y_test, model.predict(X_test)), equal_opportunity_score(sensitive_column=lsac_fair_feature)(model, X_test, y_test), p_percent_score(sensitive_column=lsac_fair_feature)(model, X_test, y_test), coefficient_of_variation(model, X_test, y_test)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/root/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

patience counter: 186 total iterations: 201
-----------------------------------------


(0.95125, 0.9927797833935018, 0.9890965732087228, 0.2915407111293814)

## 3. Adult

In [26]:
adult_data = pd.read_csv("Datasets - Limpos/adult.csv")
adult_fair_feature = "race"
adult_pred_feature = "income"

In [75]:
adult_result_metrics = kfold_methods(adult_data, adult_fair_feature, adult_pred_feature, 5000)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/p

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encounter

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encounter

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encounter

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encounter

...

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeW

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeW

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeW

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encounter

In [77]:
adult_result_metrics

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy           0.800129          0.902919  0.580725    0.702279
         equal_opportunity  0.796671          0.835233  0.507027    0.708816
         p_percent           0.79499           0.91171  0.607574    0.711518
         c_variation         0.79777          0.875411  0.545714    0.704447
RegEqual accuracy           0.833678          0.684414  0.407508    0.606748
         equal_opportunity  0.823691          0.577969  0.348672    0.642833
         p_percent          0.818035          0.631938  0.395307    0.659649
         c_variation        0.833613          0.681452  0.403632    0.603378
RegDemo  accuracy           0.833775          0.691951  0.414395     0.60538
         equal_opportunity  0.822818          0.753691   0.52616    0.637118
         p_percent          0.789593          0.942627  0.842684    0.714802
         c_variation        0.833452           0.68615  0.405625    0.603752
RegRewe  accuracy            0.79554          0.875553  0.537218    0.706152
         equal_opportunity  0.795249          0.859786  0.556886    0.709942
         p_percent          0.795281           0.87802  0.582446    0.709725
         c_variation         0.79457          0.852021  0.534675    0.706681
RegMoo   accuracy           0.799483          0.952546  0.619571    0.706299
         equal_opportunity  0.795572          0.917806  0.594572    0.711143
         p_percent          0.795992          0.915653  0.574203    0.710996
         c_variation        0.796477          0.894589  0.583932    0.707537
RegEqMoo accuracy           0.795443          0.878019  0.582497    0.708879
         equal_opportunity  0.241435                 1         1    0.344154
         p_percent          0.241435                 1         1    0.344154
         c_variation        0.241435                 1         1    0.344154

In [83]:
adult_result_ensemble = kfold_ensemble(adult_data, adult_fair_feature, adult_pred_feature, 5000)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/p

In [84]:
adult_result_ensemble

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft  0.795895          0.900096  0.556878    0.712475
         voting hard  0.795507          0.913082  0.555397    0.713257
         knorau       0.795637           0.91614  0.558075    0.712947
         knorae       0.798707          0.968673  0.604382    0.707138
RegEqMoo voting soft  0.571849          0.910869  0.739753    0.568572
         voting hard  0.572107          0.908147  0.732402    0.568269
         knorau       0.693374          0.874981  0.689004    0.684818
         knorae       0.701778          0.930457  0.840158    0.763336

In [27]:
adult_data = adult_data.drop(['Unnamed: 0'], axis=1)
X = adult_data.drop(adult_pred_feature, axis=1)
y = adult_data[adult_pred_feature]

X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=4000)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=4000)

a_train = X_train[adult_fair_feature]
a_val = X_val[adult_fair_feature]

a_train[a_train==0] = -1
a_val[a_val==0] = -1

C = 10**2
model = SKLearn_Weighted_LLR(X_train.values, y_train.values,
                             a_train.values, X_val.values,
                             y_val.values, a_val.values,
                             C_reg=C)

mua_ini = np.ones(a_val.max() + 1)
mua_ini /= mua_ini.sum()
results = APSTAR(model, mua_ini, niter=200, max_patience=200, Kini=1,
                      Kmin=20, alpha=0.5, verbose=False)
mu_best_list = results['mu_best_list']

mu_best = mu_best_list[-1]
model.weighted_fit(X_train.values, y_train.values, a_train.values, mu_best)

accuracy_score(y_test, model.predict(X_test)), equal_opportunity_score(sensitive_column=adult_fair_feature)(model, X_test, y_test), p_percent_score(sensitive_column=adult_fair_feature)(model, X_test, y_test), coefficient_of_variation(model, X_test, y_test)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/root/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

TypeError: 'float' object cannot be interpreted as an integer

## 4. COMPAS

In [33]:
compas_data = pd.read_csv("Datasets - Limpos/compas_onerace.csv")
compas_fair_feature = "not_white"
compas_pred_feature = "Two_yr_Recidivism"

In [7]:
compas_result_metrics = kfold_methods(compas_data, compas_fair_feature, compas_pred_feature, 1000)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid 

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid 

...

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid 

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for not_white == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid 

In [8]:
compas_result_metrics

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy           0.684534          0.550834  0.435784    0.886792
         equal_opportunity  0.645506          0.723987  0.541259    0.994532
         p_percent          0.664453          0.619876   0.51374    0.884612
         c_variation        0.663482          0.590743  0.505461    0.868211
RegEqual accuracy           0.671417           0.59289  0.502576    0.884696
         equal_opportunity  0.663158          0.630246  0.493689     0.94455
         p_percent          0.668826          0.622562   0.51139    0.888952
         c_variation         0.66251          0.594964  0.508957    0.865776
RegDemo  accuracy           0.672227          0.593389  0.499494    0.881448
         equal_opportunity  0.562105          0.774746   0.67572     1.10951
         p_percent          0.564372          0.954972  0.858001     1.14032
         c_variation         0.66251          0.594964  0.508957    0.865776
RegRewe  accuracy           0.662348          0.837629  0.756421    0.902274
         equal_opportunity  0.670607          0.920423  0.838623    0.907251
         p_percent           0.66753          0.893206  0.812573    0.904176
         c_variation        0.653279          0.835697  0.760055    0.887894
RegMoo   accuracy           0.683077          0.552837  0.424726    0.891024
         equal_opportunity  0.661053          0.879783  0.804195    0.835371
         p_percent          0.661053          0.879783  0.804195    0.835371
         c_variation        0.660891          0.819909   0.75083    0.843515
RegEqMoo accuracy           0.662996          0.599397  0.515107    0.868982
         equal_opportunity  0.446964                 1         1    0.452737
         p_percent          0.446964                 1         1    0.452737
         c_variation        0.446964                 1         1    0.452737

In [34]:
compas_result_ensemble = kfold_ensemble(compas_data, compas_fair_feature, compas_pred_feature,1000)

2020-10-20 22:42:02,946 - moopt.monise - DEBUG: Finding 1th individual minima
2020-10-20 22:42:03,048 - moopt.monise - DEBUG: Finding 2th individual minima
2020-10-20 22:42:03,137 - moopt.monise - DEBUG: Finding 3th individual minima
2020-10-20 22:42:03,258 - moopt.monise - DEBUG: 4th solution - importance: 1.0
2020-10-20 22:42:03,344 - moopt.monise - DEBUG: 5th solution - importance: 1.0
2020-10-20 22:42:03,458 - moopt.monise - DEBUG: 6th solution - importance: 1.0
2020-10-20 22:42:03,638 - moopt.monise - DEBUG: 7th solution - importance: 1.0
2020-10-20 22:42:03,771 - moopt.monise - DEBUG: 8th solution - importance: 1.0
2020-10-20 22:42:04,010 - moopt.monise - DEBUG: 9th solution - importance: 0.2832080734909217
2020-10-20 22:42:04,175 - moopt.monise - DEBUG: 10th solution - importance: 0.27123011667941427
2020-10-20 22:42:04,382 - moopt.monise - DEBUG: 11th solution - importance: 0.2566717167050636
2020-10-20 22:42:04,638 - moopt.monise - DEBUG: 12th solution - importance: 0.14055481

2020-10-20 22:43:29,108 - moopt.monise - DEBUG: 87th solution - importance: 0.006553200083648319
2020-10-20 22:43:31,365 - moopt.monise - DEBUG: 88th solution - importance: 0.006175531966547125
2020-10-20 22:43:33,628 - moopt.monise - DEBUG: 89th solution - importance: 0.005989436906150895
2020-10-20 22:43:35,891 - moopt.monise - DEBUG: 90th solution - importance: 0.005989436906150895
2020-10-20 22:43:38,152 - moopt.monise - DEBUG: 91th solution - importance: 0.005989436906150895
2020-10-20 22:43:40,417 - moopt.monise - DEBUG: 92th solution - importance: 0.005989436906150895
2020-10-20 22:43:42,669 - moopt.monise - DEBUG: 93th solution - importance: 0.0058730124710593854
2020-10-20 22:43:44,924 - moopt.monise - DEBUG: 94th solution - importance: 0.0058730124710593854
2020-10-20 22:43:47,203 - moopt.monise - DEBUG: 95th solution - importance: 0.0058730124710593854
2020-10-20 22:43:49,479 - moopt.monise - DEBUG: 96th solution - importance: 0.0058730124710593854
2020-10-20 22:43:51,768 - 

2020-10-20 22:46:12,573 - moopt.monise - DEBUG: Finding 2th individual minima
2020-10-20 22:46:12,661 - moopt.monise - DEBUG: Finding 3th individual minima
2020-10-20 22:46:12,752 - moopt.monise - DEBUG: 4th solution - importance: 1.0
2020-10-20 22:46:12,820 - moopt.monise - DEBUG: 5th solution - importance: 1.0
2020-10-20 22:46:12,919 - moopt.monise - DEBUG: 6th solution - importance: 1.0
2020-10-20 22:46:13,046 - moopt.monise - DEBUG: 7th solution - importance: 1.0
2020-10-20 22:46:13,168 - moopt.monise - DEBUG: 8th solution - importance: 1.0
2020-10-20 22:46:13,317 - moopt.monise - DEBUG: 9th solution - importance: 0.35327402427229765
2020-10-20 22:46:13,499 - moopt.monise - DEBUG: 10th solution - importance: 0.2803903421392772
2020-10-20 22:46:13,671 - moopt.monise - DEBUG: 11th solution - importance: 0.20416878502115066
2020-10-20 22:46:13,957 - moopt.monise - DEBUG: 12th solution - importance: 0.15478114176244
2020-10-20 22:46:14,144 - moopt.monise - DEBUG: 13th solution - import

2020-10-20 22:47:37,278 - moopt.monise - DEBUG: 88th solution - importance: 0.005756652951399231
2020-10-20 22:47:39,539 - moopt.monise - DEBUG: 89th solution - importance: 0.005648261077843261
2020-10-20 22:47:41,800 - moopt.monise - DEBUG: 90th solution - importance: 0.005633112798141573
2020-10-20 22:47:44,063 - moopt.monise - DEBUG: 91th solution - importance: 0.005577381216537913
2020-10-20 22:47:46,355 - moopt.monise - DEBUG: 92th solution - importance: 0.005450303785082691
2020-10-20 22:47:48,602 - moopt.monise - DEBUG: 93th solution - importance: 0.005490927106057305
2020-10-20 22:47:50,872 - moopt.monise - DEBUG: 94th solution - importance: 0.005490927106057305
2020-10-20 22:47:53,132 - moopt.monise - DEBUG: 95th solution - importance: 0.005490927106057305
2020-10-20 22:47:55,404 - moopt.monise - DEBUG: 96th solution - importance: 0.005490927106057305
2020-10-20 22:47:57,709 - moopt.monise - DEBUG: 97th solution - importance: 0.005490927106057305
2020-10-20 22:48:00,003 - moop

2020-10-20 22:50:18,816 - moopt.monise - DEBUG: Finding 3th individual minima
2020-10-20 22:50:18,910 - moopt.monise - DEBUG: 4th solution - importance: 1.0
2020-10-20 22:50:19,006 - moopt.monise - DEBUG: 5th solution - importance: 1.0
2020-10-20 22:50:19,156 - moopt.monise - DEBUG: 6th solution - importance: 1.0
2020-10-20 22:50:19,286 - moopt.monise - DEBUG: 7th solution - importance: 1.0
2020-10-20 22:50:19,441 - moopt.monise - DEBUG: 8th solution - importance: 1.0
2020-10-20 22:50:19,593 - moopt.monise - DEBUG: 9th solution - importance: 0.4847829580313633
2020-10-20 22:50:19,807 - moopt.monise - DEBUG: 10th solution - importance: 0.28258547259365696
2020-10-20 22:50:20,000 - moopt.monise - DEBUG: 11th solution - importance: 0.18930801994557847
2020-10-20 22:50:20,199 - moopt.monise - DEBUG: 12th solution - importance: 0.17517380332778648
2020-10-20 22:50:20,449 - moopt.monise - DEBUG: 13th solution - importance: 0.14056905233223996
2020-10-20 22:50:20,685 - moopt.monise - DEBUG: 1

2020-10-20 22:51:42,350 - moopt.monise - DEBUG: 88th solution - importance: 0.007658620710006215
2020-10-20 22:51:44,628 - moopt.monise - DEBUG: 89th solution - importance: 0.007222688511372686
2020-10-20 22:51:46,909 - moopt.monise - DEBUG: 90th solution - importance: 0.007222688511372686
2020-10-20 22:51:49,168 - moopt.monise - DEBUG: 91th solution - importance: 0.007105790825240633
2020-10-20 22:51:51,438 - moopt.monise - DEBUG: 92th solution - importance: 0.007100649803689607
2020-10-20 22:51:53,715 - moopt.monise - DEBUG: 93th solution - importance: 0.007100649803689607
2020-10-20 22:51:55,974 - moopt.monise - DEBUG: 94th solution - importance: 0.007100649803689607
2020-10-20 22:51:58,243 - moopt.monise - DEBUG: 95th solution - importance: 0.007100649803689607
2020-10-20 22:52:00,502 - moopt.monise - DEBUG: 96th solution - importance: 0.006784427534952746
2020-10-20 22:52:02,791 - moopt.monise - DEBUG: 97th solution - importance: 0.006246178089406361
2020-10-20 22:52:05,064 - moop

2020-10-20 22:54:23,755 - moopt.monise - DEBUG: 8th solution - importance: 1.0
2020-10-20 22:54:23,918 - moopt.monise - DEBUG: 9th solution - importance: 0.3566787936796257
2020-10-20 22:54:24,121 - moopt.monise - DEBUG: 10th solution - importance: 0.2913908109437618
2020-10-20 22:54:24,290 - moopt.monise - DEBUG: 11th solution - importance: 0.2695387209168142
2020-10-20 22:54:24,502 - moopt.monise - DEBUG: 12th solution - importance: 0.15534493482525405
2020-10-20 22:54:24,690 - moopt.monise - DEBUG: 13th solution - importance: 0.1473540158139031
2020-10-20 22:54:24,911 - moopt.monise - DEBUG: 14th solution - importance: 0.10508968281841016
2020-10-20 22:54:25,122 - moopt.monise - DEBUG: 15th solution - importance: 0.10070036389132432
2020-10-20 22:54:25,361 - moopt.monise - DEBUG: 16th solution - importance: 0.09291738919040397
2020-10-20 22:54:25,626 - moopt.monise - DEBUG: 17th solution - importance: 0.07268379595812967
2020-10-20 22:54:25,931 - moopt.monise - DEBUG: 18th solution 

2020-10-20 22:55:58,220 - moopt.monise - DEBUG: 92th solution - importance: 0.006648443429627187
2020-10-20 22:56:00,471 - moopt.monise - DEBUG: 93th solution - importance: 0.006400514505993454
2020-10-20 22:56:02,742 - moopt.monise - DEBUG: 94th solution - importance: 0.006356274494940312
2020-10-20 22:56:05,010 - moopt.monise - DEBUG: 95th solution - importance: 0.006356274494940312
2020-10-20 22:56:07,269 - moopt.monise - DEBUG: 96th solution - importance: 0.006356274494940312
2020-10-20 22:56:09,531 - moopt.monise - DEBUG: 97th solution - importance: 0.006545261663658082
2020-10-20 22:56:11,807 - moopt.monise - DEBUG: 98th solution - importance: 0.006545261663658082
2020-10-20 22:56:14,077 - moopt.monise - DEBUG: 99th solution - importance: 0.006545261663658082
2020-10-20 22:56:16,347 - moopt.monise - DEBUG: 100th solution - importance: 0.006545261663658082
2020-10-20 22:56:18,627 - moopt.monise - DEBUG: 101th solution - importance: 0.006545261663658082
2020-10-20 22:56:20,917 - mo

2020-10-20 22:58:38,566 - moopt.monise - DEBUG: 12th solution - importance: 0.17340672426307835
2020-10-20 22:58:38,790 - moopt.monise - DEBUG: 13th solution - importance: 0.13564145140880757
2020-10-20 22:58:39,012 - moopt.monise - DEBUG: 14th solution - importance: 0.1170547228006066
2020-10-20 22:58:39,271 - moopt.monise - DEBUG: 15th solution - importance: 0.11319819374590942
2020-10-20 22:58:39,489 - moopt.monise - DEBUG: 16th solution - importance: 0.09292651588555097
2020-10-20 22:58:39,751 - moopt.monise - DEBUG: 17th solution - importance: 0.0738902854299546
2020-10-20 22:58:40,054 - moopt.monise - DEBUG: 18th solution - importance: 0.0681464911109643
2020-10-20 22:58:40,323 - moopt.monise - DEBUG: 19th solution - importance: 0.06263594199138764
2020-10-20 22:58:40,609 - moopt.monise - DEBUG: 20th solution - importance: 0.059443852972806056
2020-10-20 22:58:40,992 - moopt.monise - DEBUG: 21th solution - importance: 0.052394154771636045
2020-10-20 22:58:41,332 - moopt.monise - 

2020-10-20 23:00:36,845 - moopt.monise - DEBUG: 96th solution - importance: 0.00633842666242048
2020-10-20 23:00:39,189 - moopt.monise - DEBUG: 97th solution - importance: 0.006486437636026023
2020-10-20 23:00:41,572 - moopt.monise - DEBUG: 98th solution - importance: 0.006486437636026023
2020-10-20 23:00:43,973 - moopt.monise - DEBUG: 99th solution - importance: 0.006486437636026023
2020-10-20 23:00:46,364 - moopt.monise - DEBUG: 100th solution - importance: 0.006486437636026023
2020-10-20 23:00:48,684 - moopt.monise - DEBUG: 101th solution - importance: 0.006486437636026023
2020-10-20 23:00:51,035 - moopt.monise - DEBUG: 102th solution - importance: 0.005735112333797151
2020-10-20 23:00:53,388 - moopt.monise - DEBUG: 103th solution - importance: 0.005735112333797151
2020-10-20 23:00:55,679 - moopt.monise - DEBUG: 104th solution - importance: 0.005847343081915702
2020-10-20 23:00:58,014 - moopt.monise - DEBUG: 105th solution - importance: 0.005847343081915702
2020-10-20 23:01:00,321 -

In [35]:
compas_result_ensemble

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft  0.670607          0.642746  0.479856    0.945242
         voting hard  0.675951          0.616245  0.457831    0.932609
         knorau       0.684372            0.5927  0.455719    0.906375
         knorae       0.646478          0.661794  0.541131    0.954363
RegEqMoo voting soft  0.446964                 1         1    0.452737
         voting hard  0.446964                 1         1    0.452737
         knorau       0.557571          0.922434  0.885182    0.615241
         knorae        0.58753          0.748565  0.721719    0.891054

In [31]:
compas_data = compas_data.drop(['Unnamed: 0'], axis=1)
X = compas_data.drop(compas_pred_feature, axis=1)
y = compas_data[compas_pred_feature]

X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=1000)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=1000)

a_train = X_train[compas_fair_feature]
a_val = X_val[compas_fair_feature]

a_train[a_train==0] = -1
a_val[a_val==0] = -1

C = 10**2
model = SKLearn_Weighted_LLR(X_train.values, y_train.values,
                             a_train.values, X_val.values,
                             y_val.values, a_val.values,
                             C_reg=C)

mua_ini = np.ones(a_val.max() + 1)
mua_ini /= mua_ini.sum()
results = APSTAR(model, mua_ini, niter=200, max_patience=200, Kini=1,
                      Kmin=20, alpha=0.5, verbose=False)
mu_best_list = results['mu_best_list']

mu_best = mu_best_list[-1]
model.weighted_fit(X_train.values, y_train.values, a_train.values, mu_best)

accuracy_score(y_test, model.predict(X_test)), equal_opportunity_score(sensitive_column=compas_fair_feature)(model, X_test, y_test), p_percent_score(sensitive_column=compas_fair_feature)(model, X_test, y_test), coefficient_of_variation(model, X_test, y_test)

patience counter: 66 total iterations: 201
-----------------------------------------


(0.633, 0.5711599482900458, 0.5180240320427236, 1.0153493459112246)